Python 3.11 has about 45-50% improvement over previous version for simple calculations like fibonacci function

In [1]:
!apt install -y -qq python3.7
!apt install -y -qq python3.8
!apt install -y -qq python3.9
!apt install -y -qq python3.10
!apt install -y -qq python3.11

python3.7 is already the newest version (3.7.15-1+bionic1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8 python3.8-minimal
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,693 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Selecting previously unselected package libpython3.8-minimal:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to un

In [2]:
script = """
import platform
import json
import os
import time

N = 30
n_sim = 100

def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

def bench(n):
    t1 = time.time()
    fibonacci(n)
    t2 = time.time()
    return t2-t1

try:
    with open("results.json", "r") as f:
        results = json.load(f)
except Exception:
    results = []

for _ in range(n_sim):
    duration = bench(N)
    result = {"version": platform.python_version(), "n": N, "duration": duration}
    results.append(result)

with open("results.json", "w") as f:
    json.dump(fp=f, obj=results, indent=4)
"""

with open("script.py", "w") as f:
    f.write(script)

In [3]:
!python3.7 script.py
!python3.8 script.py
!python3.9 script.py
!python3.10 script.py
!python3.11 script.py

In [5]:
# get the exact version for python3.11
latest, = !python3.11 -c "import platform; print(platform.python_version())"


In [6]:
import pandas as pd
df = pd.read_json("results.json")

In [7]:
df_grouped = df[['version', 'duration']].groupby(["version"])['duration'].agg(['count','mean'])
df_grouped


latest_mean = df_grouped.loc[latest]['mean']
df_grouped['latest_improvement'] = 1 - latest_mean / df_grouped['mean']
df_grouped = df_grouped.reset_index()
df_grouped[['version_major', 'version_minor', 'version_patch']] = df_grouped['version'].str.split('.', expand=True).astype(int)
df_grouped = df_grouped.sort_values(['version_major', 'version_minor', 'version_patch'])\
    .drop(columns=['version_major', 'version_minor', 'version_patch'])\
    .reset_index(drop=True)
df_grouped

,version,count,mean,latest_improvement
0,3.7.15,100,0.321914,0.457097
1,3.8.15,100,0.343422,0.491098
2,3.9.15,100,0.337367,0.481966
3,3.10.8,100,0.338496,0.483692
4,3.11.0,100,0.174768,0.000000


In [8]:
df_grouped

,version,count,mean,latest_improvement
0,3.7.15,100,0.321914,0.457097
1,3.8.15,100,0.343422,0.491098
2,3.9.15,100,0.337367,0.481966
3,3.10.8,100,0.338496,0.483692
4,3.11.0,100,0.174768,0.000000


In [11]:
df_grouped.to_markdown("README.md")